In [2]:
from census import Census
import pandas as pd
import numpy as np

api_key = "85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98"

c = Census(api_key, year=2016)
c

In [15]:
a = c.acs5.tables()
state_data = c.acs5.get(("NAME","B01003_001E"),{'for':'state:*'})
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E","B23025_005E",
                          "B17001_002E"), {'for': 'county:*'})

state_pd = pd.DataFrame(state_data)
census_pd = pd.DataFrame(census_data)

state_pd = state_pd.rename(columns={"B01003_001E":"Population",
                                   "NAME":"Name",
                                   "state":"State"})
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployed Citizens",
                                      "NAME": "Name", "county": "County"})

census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

state_pd = state_pd[["State","Name","Population"]]
census_pd = census_pd[["County","Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployed Citizens"]]
print(len(state_pd))
print(len(census_pd))

52
3220


In [16]:
census_pd.head()

,County,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployed Citizens
0,015,"Carroll County, Arkansas",27690.0,44.4,38145.0,20888.0,4789.0,17.295052,739.0
1,017,"Chicot County, Arkansas",11189.0,43.7,29628.0,18853.0,3252.0,29.064260,446.0
2,019,"Clark County, Arkansas",22684.0,33.6,35595.0,19500.0,4762.0,20.992770,858.0
3,021,"Clay County, Arkansas",15202.0,44.0,32404.0,19192.0,3340.0,21.970793,474.0
4,023,"Cleburne County, Arkansas",25587.0,47.9,41717.0,25078.0,3901.0,15.246023,903.0


In [17]:
state_pd.head()

,State,Name,Population
0,01,Alabama,4841164.0
1,02,Alaska,736855.0
2,04,Arizona,6728577.0
3,05,Arkansas,2968472.0
4,06,California,38654206.0


In [18]:
new = census_pd["Name"].str.split(",", n = 1, expand = True)
census_pd["County"] = new[0]
census_pd['State'] = new[1]
census_pd['State'].value_counts()
census_pd.head()
census_pd.drop(columns='Name')

,County,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployed Citizens,State
0,Carroll County,27690.0,44.4,38145.0,20888.0,4789.0,17.295052,739.0,Arkansas
1,Chicot County,11189.0,43.7,29628.0,18853.0,3252.0,29.064260,446.0,Arkansas
2,Clark County,22684.0,33.6,35595.0,19500.0,4762.0,20.992770,858.0,Arkansas
3,Clay County,15202.0,44.0,32404.0,19192.0,3340.0,21.970793,474.0,Arkansas
4,Cleburne County,25587.0,47.9,41717.0,25078.0,3901.0,15.246023,903.0,Arkansas
5,Cleveland County,8421.0,42.5,42429.0,21485.0,1671.0,19.843249,317.0,Arkansas
6,Columbia County,24168.0,36.0,36507.0,19922.0,5581.0,23.092519,1044.0,Arkansas
7,Conway County,21060.0,41.0,38266.0,21896.0,4482.0,21.282051,650.0,Arkansas
8,Craighead County,102884.0,33.9,43892.0,24226.0,18682.0,18.158314,3182.0,Arkansas
9,Crawford County,61927.0,38.8,41792.0,20889.0,11491.0,18.555719,1940.0,Arkansas


In [19]:
census_pd.head()

,County,Name,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployed Citizens,State
0,Carroll County,"Carroll County, Arkansas",27690.0,44.4,38145.0,20888.0,4789.0,17.295052,739.0,Arkansas
1,Chicot County,"Chicot County, Arkansas",11189.0,43.7,29628.0,18853.0,3252.0,29.064260,446.0,Arkansas
2,Clark County,"Clark County, Arkansas",22684.0,33.6,35595.0,19500.0,4762.0,20.992770,858.0,Arkansas
3,Clay County,"Clay County, Arkansas",15202.0,44.0,32404.0,19192.0,3340.0,21.970793,474.0,Arkansas
4,Cleburne County,"Cleburne County, Arkansas",25587.0,47.9,41717.0,25078.0,3901.0,15.246023,903.0,Arkansas


In [20]:
census_pd = census_pd[["County","State", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployed Citizens"]]

In [45]:
census_pd["Unemployment Rate (%)"] = census_pd["Unemployed Citizens"]/census_pd["Population"]*100
census_pd.head()

,County,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployed Citizens,Unemployment Rate,Unemployment Rate (%)
0,Carroll County,Arkansas,27690.0,44.4,38145.0,20888.0,4789.0,17.295052,739.0,2.668834,2.668834
1,Chicot County,Arkansas,11189.0,43.7,29628.0,18853.0,3252.0,29.064260,446.0,3.986058,3.986058
2,Clark County,Arkansas,22684.0,33.6,35595.0,19500.0,4762.0,20.992770,858.0,3.782402,3.782402
3,Clay County,Arkansas,15202.0,44.0,32404.0,19192.0,3340.0,21.970793,474.0,3.118011,3.118011
4,Cleburne County,Arkansas,25587.0,47.9,41717.0,25078.0,3901.0,15.246023,903.0,3.529136,3.529136


In [21]:
census_pd.to_csv("CensusDataFrame.csv")

In [41]:
census_pd[census_pd["County"] == "Franklin County"]

,County,State,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Unemployed Citizens
16,Franklin County,Arkansas,17799.0,41.0,39482.0,19811.0,3684.0,20.697792,638.0
91,Franklin County,Alabama,31573.0,39.1,36138.0,18193.0,6951.0,22.015646,1323.0
337,Franklin County,Florida,11705.0,43.8,40301.0,21515.0,2060.0,17.599317,437.0
445,Franklin County,Georgia,22188.0,40.7,37298.0,18642.0,5426.0,24.454660,691.0
571,Franklin County,Idaho,13013.0,32.8,50500.0,19985.0,1606.0,12.341505,220.0
622,Franklin County,Illinois,39503.0,41.9,39507.0,22119.0,7440.0,18.834013,1861.0
782,Franklin County,Indiana,22887.0,42.1,52128.0,24748.0,2591.0,11.320837,698.0
885,Franklin County,Iowa,10393.0,42.5,47524.0,24909.0,1509.0,14.519388,279.0
979,Franklin County,Kansas,25663.0,39.3,51465.0,24597.0,3357.0,13.081090,801.0
1091,Franklin County,Kentucky,49982.0,40.8,50999.0,27252.0,7054.0,14.113081,1951.0


In [34]:
census_pd['State'].value_counts()

 Texas                   254
 Georgia                 159
 Virginia                133
 Kentucky                120
 Missouri                115
 Kansas                  105
 Illinois                102
 North Carolina          100
 Iowa                     99
 Tennessee                95
 Nebraska                 93
 Indiana                  92
 Ohio                     88
 Minnesota                87
 Michigan                 83
 Mississippi              82
 Puerto Rico              78
 Oklahoma                 77
 Arkansas                 75
 Wisconsin                72
 Florida                  67
 Pennsylvania             67
 Alabama                  67
 South Dakota             66
 Louisiana                64
 Colorado                 64
 New York                 62
 California               58
 Montana                  56
 West Virginia            55
 North Dakota             53
 South Carolina           46
 Idaho                    44
 Washington               39
 Oregon       